# Matching CBGs to ZCTAs for COVID-19 research

The code below matches CBGs to ZCTAs in a many-to-one fashion using CBG centroids: 

    A CBG is assigned to a ZCTA if that CBG's centroid is within the ZCTA's polygon.

This strategy is not perfect but it does allow fast matching of CBG data to ZCTA level COVID-19 data. Below, ZIP code and ZCTA are used interchangeably for simplicity but ZCTAs are <i>always</i> the areal unit of analysis, except for NYC which combines ZCTAs in their COVID-19 reporting. Notably, this strategy will work for any type of polygon, thus NYC's unique data reporting can be easily accommodated.

<b>Requirements:</b>
- <a href='https://geopandas.org/'>GeoPandas</a>
- Pandas (dependency for GeoPandas)
    
Note: <a href='https://geopandas.org/install.html'>Installing GeoPandas</a> can be non-trivial in some cases due to its dependence on, for example, GDAL. However, issues are often easily resolved with a little digging.

In [2]:
import geopandas as gpd
import pandas as pd

data_dir = ''
sg_dir = data_dir+'safegraph/'
cbg_dir = sg_dir+'safegraph_open_census_data/'
covid_dir = data_dir+'covid19/'

## Previously constructed Census data

In [3]:
# Import race-ethnicity data
df_cbg_re_count = pd.read_csv(cbg_dir+'cbg_re_count.csv')
df_cbg_re_count.head()

,cbg,totpop,white,black,hisp,asian,other
0,10010201001,745,569,160,16,0,0
1,10010201002,1265,1046,104,37,9,69
2,10010202001,960,361,568,13,0,18
3,10010202002,1236,604,571,15,24,22
4,10010203001,2364,1481,515,228,27,113


In [4]:
df_cbg_re_per = pd.read_csv(cbg_dir+'cbg_re_per.csv')
df_cbg_re_per.head()

,cbg,totpop,white,black,hisp,asian,other,gsi_race
0,10010201001,745,0.763758,0.214765,0.021477,0.000000,0.000000,0.370088
1,10010201002,1265,0.826877,0.082213,0.029249,0.007115,0.054545,0.305633
2,10010202001,960,0.376042,0.591667,0.013542,0.000000,0.018750,0.507988
3,10010202002,1236,0.488673,0.461974,0.012136,0.019417,0.017799,0.546937
4,10010203001,2364,0.626481,0.217851,0.096447,0.011421,0.047800,0.548346


## Previously constructed SafeGraph site visit data for particular types of businesses

In [5]:
# Import CBG NAICS site visitation data from Safegraph
df_cbg_naics_dev = pd.read_csv(sg_dir+'cbg_naics_dev.csv')
df_cbg_naics_dev.head()

,cbg,num_devices,naics,n
0,21700008003,86,445110,5
1,21700008003,86,445120,4
2,21700008003,86,446110,4
3,21700008003,86,445110,7
4,21700008003,86,445120,4


## SafeGraph provided CBG metadata

In [6]:
# Load CBG point data 
df_cbg_point = pd.read_csv(cbg_dir+'metadata/cbg_geographic_data.csv')
df_cbg_point['cbg'] = df_cbg_point['census_block_group'].astype(int)
gdf_cbg_point = gpd.GeoDataFrame(df_cbg_point,geometry=gpd.points_from_xy(df_cbg_point.longitude,df_cbg_point.latitude))
gdf_cbg_point = gdf_cbg_point[['cbg','geometry','amount_land']]
gdf_cbg_point.head()

,cbg,geometry,amount_land
0,10010201001,POINT (-86.48961 32.46583),4254524.0
1,10010201002,POINT (-86.48969 32.48585),5568295.0
2,10010202001,POINT (-86.47497 32.48008),2058380.0
3,10010202002,POINT (-86.46977 32.46443),1283506.0
4,10010203001,POINT (-86.46079 32.48018),3866515.0


## COVID-19 data from select cities (Chicago, NYC, and San Francisco)

In [7]:
# Load ZIP code COVID-19 data (note: actually ZCTAs, and NYC reporting combines multiple ZCTAs)
gdf_covid_zip = gpd.read_file(covid_dir+'covid_zip.geojson')
gdf_covid_zip.head()

,covid_geo,zip,case_rate,death_rate,geometry
0,Chicago,60647,1331.3,76.6,"MULTIPOLYGON (((-87.67762 41.91776, -87.67761 ..."
1,Chicago,60639,3111.0,89.5,"MULTIPOLYGON (((-87.72683 41.92265, -87.72693 ..."
2,Chicago,60707,416.1,20.9,"MULTIPOLYGON (((-87.78500 41.90915, -87.78531 ..."
3,Chicago,60622,1075.9,92.8,"MULTIPOLYGON (((-87.66707 41.88885, -87.66707 ..."
4,Chicago,60651,2314.2,93.3,"MULTIPOLYGON (((-87.70656 41.89555, -87.70672 ..."


In [8]:
# How many ZIP codes per covid_geo?
gdf_covid_zip['covid_geo'].value_counts()

Chicago          60
Queens           59
Manhattan        44
Brooklyn         37
Bronx            25
San Francisco    23
Staten Island    12
Name: covid_geo, dtype: int64

## Use GeoPandas to do spatial join using 'within' criterion

In [9]:
# Make sure projections align
common_crs = gdf_covid_zip.crs
gdf_cbg_point.crs = common_crs
gdf_covid_zip.crs==gdf_cbg_point.crs

True

In [10]:
# Perform spatial join (many CBG points -> one ZIP polygon)
gdf_cbg_zip = gpd.sjoin(gdf_cbg_point,gdf_covid_zip,op='within',how='inner').reset_index()
gdf_cbg_zip.head()

,index,cbg,geometry,amount_land,index_right,covid_geo,zip,case_rate,death_rate
0,28133,60750101001,POINT (-122.40955 37.80852),557238.0,247,San Francisco,94133,103.926954,NaN
1,28134,60750101002,POINT (-122.41016 37.80509),219252.0,247,San Francisco,94133,103.926954,NaN
2,28138,60750103001,POINT (-122.41637 37.80315),129884.0,247,San Francisco,94133,103.926954,NaN
3,28139,60750103002,POINT (-122.41477 37.80094),61524.0,247,San Francisco,94133,103.926954,NaN
4,28140,60750103003,POINT (-122.41524 37.79958),76858.0,247,San Francisco,94133,103.926954,NaN


In [11]:
# With spatial operation complete, extract the crosswalk for use
df_cbg_within_zip = gdf_cbg_zip[['zip','cbg']]
df_cbg_within_zip.head()

,zip,cbg
0,94133,60750101001
1,94133,60750101002
2,94133,60750103001
3,94133,60750103002
4,94133,60750103003


In [12]:
# NOTE: Losing 3 ZIP codes somewhere
len(df_cbg_within_zip), len(df_cbg_within_zip['zip'].unique()), len(gdf_covid_zip['zip'].unique())

(9020, 255, 258)

In [14]:
# Merge COVID-19 ZIP data with CBG-ZIP crosswalk to get CBGs with COVID-19 ZIP code data
df_covid_zip = gdf_covid_zip[['covid_geo','zip','case_rate']]
df_covid_zip['city'] = df_covid_zip['covid_geo']
df_covid_zip.loc[df_covid_zip['covid_geo'].isin(['Manhattan','Queens','Bronx','Staten Island','Brooklyn']),'city'] = 'NYC'
df_covid_zip_cbg = df_cbg_within_zip.merge(df_covid_zip,on='zip')
df_covid_zip_cbg.head()

/home/graziul/.conda/envs/sg36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,zip,cbg,covid_geo,case_rate,city
0,94133,60750101001,San Francisco,103.926954,San Francisco
1,94133,60750101002,San Francisco,103.926954,San Francisco
2,94133,60750103001,San Francisco,103.926954,San Francisco
3,94133,60750103002,San Francisco,103.926954,San Francisco
4,94133,60750103003,San Francisco,103.926954,San Francisco


In [18]:
# Merge COVID-19 CBG-ZIP code data with CBG race-ethnicity data
df_covid_zip_cbg_re = df_covid_zip_cbg.merge(df_cbg_re_per,on='cbg')
df_covid_zip_cbg_re = df_covid_zip_cbg_re.merge(gdf_cbg_point[['cbg','amount_land']], on='cbg')
df_covid_zip_cbg_re['area_sqmi'] = df_covid_zip_cbg_re['amount_land']/2589988
df_covid_zip_cbg_re['popdens'] = df_covid_zip_cbg_re['totpop']/df_covid_zip_cbg_re['area_sqmi']
df_covid_zip_cbg_re = df_covid_zip_cbg_re.drop(columns=['amount_land'])
df_covid_zip_cbg_re.head()

,zip,cbg,covid_geo,case_rate,city,totpop,white,black,hisp,asian,other,gsi_race,area_sqmi,popdens
0,94133,60750101001,San Francisco,103.926954,San Francisco,1101,0.429609,0.012716,0.071753,0.485922,0.000000,0.574005,0.215151,5117.340863
1,94133,60750101002,San Francisco,103.926954,San Francisco,2899,0.475336,0.047258,0.131425,0.303898,0.042083,0.660425,0.084654,34245.412639
2,94133,60750103001,San Francisco,103.926954,San Francisco,1910,0.557068,0.000000,0.102094,0.316754,0.024084,0.578339,0.050148,38086.885837
3,94133,60750103002,San Francisco,103.926954,San Francisco,1263,0.381631,0.000000,0.262074,0.344418,0.011876,0.666910,0.023755,53168.760874
4,94133,60750103003,San Francisco,103.926954,San Francisco,1168,0.665240,0.000000,0.027397,0.276541,0.030822,0.479281,0.029675,39359.676078
